# Kinesis Data Analytics for SQL Applications

With Amazon Kinesis Data Analytics for SQL Applications, you can process and analyze streaming data using standard SQL. The service enables you to quickly author and run powerful SQL code against streaming sources to perform time series analytics, feed real-time dashboards, and create real-time metrics.

The service supports ingesting data from Amazon Kinesis Data Streams and Amazon Kinesis Data Firehose streaming sources. Then, you author your SQL code using the interactive editor and test it with live streaming data. You can also configure destinations where you want Kinesis Data Analytics to send the results.

<img src="img/use_case_1_analytics.png" width="80%" align="left">

# Create AWS Lambda Function as Kinesis Data Analytics Destination

Kinesis Data Analytics supports Amazon Kinesis Data Firehose, AWS Lambda, and Amazon Kinesis Data Streams as destinations. Let's create a Lambda function to publish our SQL application results as custom metrics to CloudWatch Metrics.

In [1]:
import boto3
import sagemaker
import pandas as pd
import json

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

iam = boto3.Session().client(service_name='iam', region_name=region)
sts = boto3.Session().client(service_name='sts', region_name=region)
account_id = sts.get_caller_identity()['Account']

lam = boto3.Session().client(service_name='lambda', region_name=region)

## Retrieve AWS Lambda Function Name

In [2]:
%store -r lambda_fn_name

In [3]:
try:
    lambda_fn_name
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [4]:
print(lambda_fn_name)

DeliverKinesisAnalyticsToCloudWatch


## Check IAM Roles Are In Place

In [5]:
%store -r iam_lambda_role_name

In [6]:
try:
    iam_lambda_role_name
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [7]:
print(iam_lambda_role_name)

DSOAWS_Lambda


In [8]:
%store -r iam_lambda_role_passed

In [9]:
try:
    iam_lambda_role_passed
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [10]:
print(iam_lambda_role_passed)

True


In [11]:
if not iam_lambda_role_passed:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')
else:
    print('[OK]')

[OK]


In [12]:
%store -r iam_role_lambda_arn

In [13]:
try:
    iam_role_lambda_arn
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [14]:
print(iam_role_lambda_arn)

arn:aws:iam::085964654406:role/DSOAWS_Lambda


# Review Lambda Function Code 

In [15]:
!pygmentize src/lambda_function.py

from __future__ import print_function
import boto3
import base64

import sys
import logging
import traceback
import json

logger = logging.getLogger()
logger.setLevel(logging.INFO)

client = boto3.client('cloudwatch')

def lambda_handler(event, context):
    output = []
    success = 0
    failure = 0
    for record in event['records']:
        try:
            #logger.info(f'event: {event}')
            payload = base64.b64decode(record['data'])
            datapoint = float(payload)
            # logger.info(f'avg_star_rating: {payload}')

            client.put_metric_data(
                Namespace='kinesis/analytics/AVGStarRating',
                MetricData=[
                    {
                        'MetricName': 'AVGStarRating',
                        'Dimensions': [
                            {
                                'Name': 'Product Category',
                                'Value': 'All'
                             },
                        ],
             

# Zip The Function Code

In [16]:
!zip src/DeliverKinesisAnalyticsToCloudWatch.zip src/lambda_function.py

  adding: src/lambda_function.py (deflated 63%)


# Load the .zip File as Binary Code

In [17]:
with open('src/DeliverKinesisAnalyticsToCloudWatch.zip', 'rb') as f: 
    code = f.read()

# Create The Lambda Function

In [18]:
from botocore.exceptions import ClientError

try: 
    response = lam.create_function(
        FunctionName='{}'.format(lambda_fn_name),
        Runtime='python2.7',
        Role='{}'.format(iam_role_lambda_arn),
        Handler='src/lambda_function.lambda_handler',
        Code={
            'ZipFile': code
        },
        Description='Deliver output records from Kinesis Analytics application to CloudWatch.',
        Timeout=60,
        MemorySize=128,
        Publish=True
    )
    print('Lambda Function {} successfully created.'.format(lambda_fn_name))

except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceConflictException':
        print('Lambda Function {} already exists. This is OK.'.format(lambda_fn_name))
    else:
        print('Error: {}'.format(e))

Lambda Function DeliverKinesisAnalyticsToCloudWatch successfully created.


In [19]:
response = lam.get_function(FunctionName=lambda_fn_name)
print(json.dumps(response, indent=4, sort_keys=True, default=str))

{
    "Code": {
        "Location": "https://awslambda-us-west-2-tasks.s3.us-west-2.amazonaws.com/snapshots/085964654406/DeliverKinesisAnalyticsToCloudWatch-870e7a4f-2bca-4f58-ae74-172d71e65d18?versionId=256nYCXe2sA8IEm7qusQMZojcHwvRHFr&X-Amz-Security-Token=IQoJb3JpZ2luX2VjENz%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLXdlc3QtMiJHMEUCIDPFmOso7%2BS%2FDoZtZQhlLn%2F9iwuV70zNthir%2B8KGkTBUAiEA9miSLQZwd7VF26rGekiP437%2FwPttFpn%2F%2Fwq32YnYtukqvQMI9f%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARACGgw1MDIyOTcwNzYxNjMiDGgvWzQ%2FeliRDMUKtCqRA2zMa0sFjbBGdeCkjOKaqPS9QrzK14I059nZM8skznxUGvgcq1nCu%2Bmzo0pcVpOgNtVFMUOI13WPaBlm9HfmgHFaoda2PWQ%2FOzofqXPjRDF5aaQfqHN1EjlzdhGDkzA8m2FVB1V1eqOfyn1pmbxoIhENtOu2Vk8W21%2BW6KDx7oP2G1eiM8%2F47RxOvdq9ctiArynppYYqLxR8%2BhhcF%2B2O%2BsY6csmOgCeKkXDGdSaTbSGSHEeQ5u93mQhrPUpeU%2BqgzGhy0bWs3opICZPz%2FVGp0W00017uOv3r9YECBgLocAbTluz4xezYvXIgSyfdP7kWf13M8LI5lScnOLMDxzYyPbyaFn8mn1u0T%2BKuWzLwJgSpTL1W7DXB1KVvUm%2B0tI4BnjcK3lY28DZFyU9ww5g72DLZfkXmCZ9gK%2FZ7xUL6fyCCKls2M2krrjlRkW%2FeQt1oeJXREVwS

In [20]:
response = lam.get_function(FunctionName=lambda_fn_name)

lambda_fn_arn = response['Configuration']['FunctionArn']
print(lambda_fn_arn)

arn:aws:lambda:us-west-2:085964654406:function:DeliverKinesisAnalyticsToCloudWatch


In [21]:
%store lambda_fn_arn

Stored 'lambda_fn_arn' (str)


# Review Lambda Function

In [22]:
from IPython.core.display import display, HTML
        
display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/lambda/home?region={}#/functions/{}"> Lambda Function</a></b>'.format(region, lambda_fn_name)))


# Store Variables for Next Notebooks

In [23]:
%store

Stored variables and their in-db values:
auto_ml_job_name                                      -> 'automl-dm-26-16-00-25'
autopilot_endpoint_name                               -> 'automl-dm-ep-26-16-21-49'
autopilot_train_s3_uri                                -> 's3://sagemaker-us-west-2-085964654406/data/amazon
balance_dataset                                       -> True
experiment_name                                       -> 'Amazon-Customer-Reviews-BERT-Experiment-160114585
firehose_arn                                          -> 'arn:aws:firehose:us-west-2:085964654406:deliverys
firehose_name                                         -> 'dsoaws-kinesis-data-firehose'
iam_kinesis_role_name                                 -> 'DSOAWS_Kinesis'
iam_kinesis_role_passed                               -> True
iam_lambda_role_name                                  -> 'DSOAWS_Lambda'
iam_lambda_role_passed                                -> True
iam_role_kinesis_arn                             

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();